In [18]:
#install needed packages for panada and widgets
%pip install ipywidgets
%pip install pandas

In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ipywidgets as widgets #widgets
from ipywidgets import Button, HBox, VBox, Layout #layout
from IPython.display import display #displays

import matplotlib.pyplot as plt

In [20]:
df= pd.read_csv('netfix_cleaned.csv') #import csv file aka the data
df # displays the DataFrame

,names,release_year,maturity_rating,duration,description,genre,mood,cast,subtitles,audio
0,Mission Majnu,2023,U/A 16+,2h 9m,"In the 1970s,an undercover Indian spy takes on...","Spy, Hindi-Language, Bollywood, Action & Adven...",Suspenseful,"Sidharth Malhotra,Rashmika Mandanna,Parmeet Se...","English,English,Hindi","English,Hindi - Audio Description,Hindi Origin..."
1,Cirkus,2022,U/A 7+,2h 14m,Chaos and comedy take the spotlight when a rin...,"Hindi-Language, Bollywood, Comedies",Goofy,"Ranveer Singh,Varun Sharma,Pooja Hegde,Jacquel...","English,Hindi",Hindi Original
2,Gangubai Kathiawadi,2022,U/A 16+,2h 33m,"Duped and sold to a brothel,a young woman fear...","Hindi-Language, Based on Books,Dramas",Provocative,"Alia Bhatt,Vijay Raaz,Seema Pahwa,Ajay Devgn,J...","English,Hindi","Hindi Original, Telugu"
3,Thunivu,2023,U/A 16+,2h 22m,A major bank heist takes an unnerving turn whe...,"Crime, Action & Adventure",Exciting,"Ajith Kumar,Manju Warrier,Samuthirakani,John K...","English,Hindi",Tamil Original
4,Bhool Bhulaiyaa 2,2022,U/A 13+,2h 21m,"When strangers Reet and Ruhan cross paths,thei...","Hindi-Language, Bollywood, Comedies,Horror",Offbeat,"Tabu,Kartik Aaryan,Kiara Advani,Rajpal Yadav,S...","English,Hindi",Hindi Original
...,...,...,...,...,...,...,...,...,...,...
555,Happy New Year,2014,U/A 13+,2h 58m,A revenge-seeking diamond thief gathers a ragt...,"Hindi-Language, Bollywood, Action & Adventure,...",Feel-Good,"Shah Rukh Khan,Deepika Padukone,Abhishek Bachc...",English,Hindi Original
556,RRR (Hindi),2022,A,3h 5m,A fearless warrior on a perilous mission comes...,"Action & Adventure,Dramas",Adrenaline Rush,"NTR Jr.,Ram Charan,Ajay Devgn,Alia Bhatt,Shriy...","English,Hindi","English,Hindi"
557,Major (Telugu),2022,U/A 16+,2h 25m,This biopic follows the life of Indian Army of...,"Based on Real Life,Action & Adventure,Dramas",Bittersweet,"Adivi Sesh,Prakash Raj,Sobhita Dhulipala,Saiee...",English,Telugu Original
558,Om Shanti Om,2007,U/A 16+,2h 48m,Reincarnated 30 years after being killed in a ...,"Romantic Comedies,Hindi-Language, Bollywood, C...",Emotional,"Shah Rukh Khan,Deepika Padukone,Shreyas Talpad...",English,Hindi Original


In [21]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#removes all english stop words
tfidf = TfidfVectorizer(stop_words='english')

#creates matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['description'])

#shape of tfidf_matrix
tfidf_matrix.shape

# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['names']).drop_duplicates()

#takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['names'].iloc[movie_indices]

get_recommendations('Mission Majnu')

155                Codename: Tiranga
278                Codename: Tiranga
120                      Interceptor
266                   Sarkar (Tamil)
375                   Sarkar (Tamil)
475                   Sarkar (Tamil)
138    Mission: Impossible - Fallout
265    Mission: Impossible - Fallout
298    Mission: Impossible - Fallout
369    Mission: Impossible - Fallout
Name: names, dtype: object

In [22]:
genres= df['genre'].tolist() #takes genres from the csv and puts them in a list
#print(df['genre'][4])

#takes list of genres from each movie and creates one list
genre_options = []
for inner_list in genres:
    new_element=inner_list.split(",")
    genre_options+=new_element

#takes list of all genres just created and finds just the unique values
unique_genre = []
for element in genre_options:
    if element not in unique_genre:
        unique_genre.append(element)
        
#print(unique_genre)

In [23]:
#creates radio buttons based on the unique genres listed in the data
genre_rb= widgets.Dropdown(
    value='Spy',
    options=unique_genre,
    layout= Layout(align_items= 'center', width='250px', vertical_align= 'middle'),
    #flex_flow='row wrap
    #description=('Movie Genres:'),
    disabled=False
)

In [24]:
# layout formatting for widgets
item_layout=Layout(display='flex', flex_flow='column',
                   border='solid 1px', align_items= 'center',
                   width='250ox', height='auto',
                   padding= '2px', vertical_align= 'middle' 
                  )

In [25]:
# creates a list of movie titles
names= df['names'].tolist()
#print(names)

#combobox for movie titles, allows user to start typing a movie and select the title
movie_cb= widgets.Combobox(
    placeholder='Start typing for suggestions',
    options= names,
    description='Enter a movie name:',
    ensure_option=True,
    disabled=False,
    layout= Layout(display='flex', flex_flow='column', align_items= 'center',
                   width='250px', height='auto', 
                   padding= '10px', vertical_align= 'middle', border='hidden'),
    style={'description_width': 'initial'}
)

In [26]:
title1= widgets.Label(value= 'Kadabra', font_weight= 'bold', 
                     layout=Layout(display='flex', flex_flow='column', border='dashed 2px', align_items= 'center',
                                   width='250px', height='auto', padding= '25px', vertical_align= 'middle'), background_color='#7b88db')
title2= widgets.Label(value= 'Movie', font_weight= 'bold', 
                     layout=Layout(display='flex', flex_flow='column', border='dashed 2px', align_items= 'center',
                                   width='250px', height='auto', padding= '25px', vertical_align= 'middle'), background_color='#7b88db')
title3= widgets.Label(value= 'Finder', font_weight= 'bold', 
                     layout=Layout(display='flex', flex_flow='column', border='dashed 2px', align_items= 'center',
                                   width='250px', height='auto', padding= '25px', vertical_align= 'middle'), background_color='#7b88db')
title_genre= widgets.Label(value= 'Movie Genres', font_weight= 'bold',
                           layout=Layout(display='flex', flex_flow='column', align_items= 'center', 
                                         height='auto', width='250px', padding= '5px', vertical_align= 'middle'), background_color='#7b88db')

In [27]:
# movie_rec = widgets.Accordion(children=[widgets.Text(),widgets.Text()], titles=('Text','Text'), layout=Layout(display='flex', flex_flow='column', border='solid 1px', align_items= 'center', 
            #                             height='auto', width='225px', padding= '5px', vertical_align= 'middle', positioning='right'))

In [28]:
# button to run recommender
button=widgets.Button(description='Find A Movie', layout=item_layout, border='solid 2px', positioning='center', align_items = 'center', vertical_align= 'middle')
#output = widgets.Output()

In [29]:
empty_label_small= widgets.Label(value= '', font_weight= 'bold', layout=Layout(display='flex', flex_flow='column', align_items= 'center',
                                                                         width='5px', height='auto', padding= '2px', vertical_align= 'middle'))

empty_label_large= widgets.Label(value= '', font_weight= 'bold', layout=Layout(display='flex', flex_flow='column', align_items= 'center',
                                                                         width='5px', height='auto', padding= '40px', vertical_align= 'middle'))

In [30]:
x= get_recommendations('Mission Majnu')
print(type(x))
for i in x:
    print(i)


<class 'pandas.core.series.Series'>
Codename: Tiranga
Codename: Tiranga
Interceptor
Sarkar (Tamil)
Sarkar (Tamil)
Sarkar (Tamil)
Mission: Impossible - Fallout
Mission: Impossible - Fallout
Mission: Impossible - Fallout
Mission: Impossible - Fallout


In [31]:
get_recommendations('Mission Majnu')

155                Codename: Tiranga
278                Codename: Tiranga
120                      Interceptor
266                   Sarkar (Tamil)
375                   Sarkar (Tamil)
475                   Sarkar (Tamil)
138    Mission: Impossible - Fallout
265    Mission: Impossible - Fallout
298    Mission: Impossible - Fallout
369    Mission: Impossible - Fallout
Name: names, dtype: object

In [34]:
output = widgets.Output()

def on_button_clicked(b):
    g= genre_rb.value
    m= movie_cb.value

    x= get_recommendations(m)
    unique_rec = []
    
    for element in x:
        if element not in unique_rec:
                unique_rec.append(element)
    with output:
        output.clear_output()
        print('hi')
        print(unique_rec, g, m)

button.on_click(on_button_clicked)

In [35]:
# widget box for radio buttons
left_box= widgets.VBox([title1,movie_cb,empty_label_small])
# widget box for combo box and button
right_box= widgets.VBox([title2,empty_label_small, title_genre, genre_rb, empty_label_small, empty_label_small, empty_label_small, empty_label_small, button])
# empty widget box for space
center_box= widgets.VBox([empty_label_small])

# widget box for recommendations
far_right_box= widgets.VBox([title3, output])

# widget box to hold other boxes
widgets.Box([left_box, right_box, far_right_box])
# df[['names', 'cast', 'genre', 'subtitles', 'audio']].head(3)

Box(children=(VBox(children=(Label(value='Kadabra', layout=Layout(align_items='center', border_bottom='dashed …

In [17]:
features = ['cast', 'genre', 'subtitles', 'audio']
# converts strings with commas into lists
for feature in features:
    df[feature]=df[feature].str.split(',')

In [69]:
counts=dict()
for i in df.index:
    for g in df.loc[i, 'genre'].split(","):
        if g not in counts:
            counts[g]=1
        else:
            counts[g]= counts[g]+1

plt.rcParams.update({'font.size': 8})
plt.bar(list(counts.keys()), counts.values())
plt.xticks(rotation=90)
plt.xlabel('Genres')
plt.ylabel('Counts')

<class 'AttributeError'>: 'list' object has no attribute 'split'